## Imports and function definitions

In [2]:
import numpy as np


In [3]:
def get_Markov_chain_matrix(p_one, exp_len_one):
    """Create a transition probability matrix for a Markov chain with two states,
    and equilibrium probbaility of state 1 being p_one,
    and expected length of staying in state 1 being exp_len_one"""
    exp_len_zero = exp_len_one / p_one * (1 - p_one)
    assert exp_len_one >= 1 and exp_len_zero >= 1 
    p_switch_one = 1 / exp_len_one
    p_switch_zero = 1 / exp_len_zero
    P = np.array([[1-p_switch_zero, p_switch_zero], [p_switch_one, 1-p_switch_one]])
    return P

def get_stationary(P):
    """Compute the stationary distribution of a Markov chain with two states."""
    p_one = P[0,1] / (P[0,1] + P[1,0])
    return np.array([1-p_one, p_one])



In [4]:
def generate_independent_samples(p_one, n):
    return np.random.binomial(1, p_one, n)

def generate_markov_chain_samples(P, n):
    pi = get_stationary(P)
    first_state = np.random.choice([0, 1], p=pi)
    states = [first_state]
    for _ in range(n - 1):
        current_state = states[-1]
        next_state = np.random.choice([0, 1], p=P[current_state])
        states.append(next_state)
    return np.array(states)

In [5]:
def compute_lengths(X, which_value):
    """Finds continuous regions with value which_value in list X and computes their lengths"""
    lengths = []
    current_length = 0
    for value in X:
        if value == which_value:
            current_length += 1
        else:
            if current_length > 0:
                lengths.append(current_length)
            current_length = 0
    if current_length > 0:
        lengths.append(current_length)
    return lengths

## Create three Markov chains

In [6]:
# create an example Markov chain
p_one = 0.7  # probability of one
matrices = [get_Markov_chain_matrix(p_one, 10), 
     get_Markov_chain_matrix(p_one, 2.5), 
     get_Markov_chain_matrix(p_one, 10/3)]
display(matrices)


[array([[0.76666667, 0.23333333],
        [0.1       , 0.9       ]]),
 array([[0.06666667, 0.93333333],
        [0.4       , 0.6       ]]),
 array([[0.3, 0.7],
        [0.3, 0.7]])]

## Create samples from Markov chain and also idependent trials

In [7]:
n = 20000    # number of samples to generate
X_indep = generate_independent_samples(p_one, n)
X_markov = [ generate_markov_chain_samples(M, n) for M in matrices]


In [8]:
m = 200     # number of samples to print
print(f"First {m} elements in the independendent and Markov sequences:")
print(X_indep[:m])
for X in X_markov:
    print(X[:m])


First 200 elements in the independendent and Markov sequences:
[1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 0 1 0 0 1
 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 0 1
 1 1 0 0 1 1 1 0 1 0 1 1 1 0 1 0 0 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 0 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0 0 1 0 1 1
 1 1 1 0 0 1 0 1 0 1 0 0 1 0 0]
[1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0
 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1]
[1 1 0 1 0 1 0 1 0 1 1 0 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 0
 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 0 1 1 0 1 1

In [9]:
print("Fraction of 1s in the independent and Markov sequences:")
print(sum(X_indep / n))
for X in X_markov:
    print(sum(X / n))

Fraction of 1s in the independent and Markov sequences:
0.6982499999999313
0.7088999999999301
0.6991999999999312
0.6965499999999315


In [10]:
for which in [0, 1]:
    print(f"Mean length of a region of {which}s in the independent and Markov sequences:")
    l_indep = compute_lengths(X_indep, which)
    print(np.mean(l_indep))
    for X in X_markov:
        l_markov = compute_lengths(X, which)
        print(np.mean(l_markov))

Mean length of a region of 0s in the independent and Markov sequences:
1.424356856266226
4.197548666186013
1.072561954002496
1.4344126683999054
Mean length of a region of 1s in the independent and Markov sequences:
3.295964125560538
10.214697406340058
2.492691622103387
3.2926022216969986


## Stationary distribution

In [11]:
# Compute powers of the matrix P
for k in [0,1,2,3,10,30]:
    power = np.linalg.matrix_power(matrices[0], k)
    display(f"P^{k}:")
    display(power)

'P^0:'

array([[1., 0.],
       [0., 1.]])

'P^1:'

array([[0.76666667, 0.23333333],
       [0.1       , 0.9       ]])

'P^2:'

...

array([[0.50740741, 0.49259259],
       [0.21111111, 0.78888889]])

'P^10:'

array([[0.31213907, 0.68786093],
       [0.29479754, 0.70520246]])

'P^30:'

array([[0.30000365, 0.69999635],
       [0.29999844, 0.70000156]])

## A simple discrete substitution matrix

In [14]:
S1 = 0.96 * np.identity(4) + np.full((4, 4), 0.01)
display(S1)

array([[0.97, 0.01, 0.01, 0.01],
       [0.01, 0.97, 0.01, 0.01],
       [0.01, 0.01, 0.97, 0.01],
       [0.01, 0.01, 0.01, 0.97]])

In [15]:
for k in [0,1,2,3,10,30,100]:
    power = np.linalg.matrix_power(S1, k)
    display(f"S({k}):")
    display(power)

'S(0):'

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

'S(1):'

array([[0.97, 0.01, 0.01, 0.01],
       [0.01, 0.97, 0.01, 0.01],
       [0.01, 0.01, 0.97, 0.01],
       [0.01, 0.01, 0.01, 0.97]])

'S(2):'

...

array([[0.74862448, 0.08379184, 0.08379184, 0.08379184],
       [0.08379184, 0.74862448, 0.08379184, 0.08379184],
       [0.08379184, 0.08379184, 0.74862448, 0.08379184],
       [0.08379184, 0.08379184, 0.08379184, 0.74862448]])

'S(30):'

array([[0.47039323, 0.17653559, 0.17653559, 0.17653559],
       [0.17653559, 0.47039323, 0.17653559, 0.17653559],
       [0.17653559, 0.17653559, 0.47039323, 0.17653559],
       [0.17653559, 0.17653559, 0.17653559, 0.47039323]])

'S(100):'

array([[0.26265274, 0.24578242, 0.24578242, 0.24578242],
       [0.24578242, 0.26265274, 0.24578242, 0.24578242],
       [0.24578242, 0.24578242, 0.26265274, 0.24578242],
       [0.24578242, 0.24578242, 0.24578242, 0.26265274]])